In [1]:
## Plot SWOT nadir altimeter data

In [2]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import matplotlib

import functions  # requires functions.py from this directory
import cartopy.crs as ccrs                   # import projections
import cartopy
import gsw


In [3]:
plt.close('all')

In [4]:
# %matplotlib inline  
%matplotlib qt5  
plt.rcParams['figure.figsize'] = (5,4)
plt.rcParams['figure.dpi'] = 200
plt.rcParams['savefig.dpi'] = 400
plt.close('all')

__figdir__ = '../plots/' + 'SSH_plots/'
savefig_args = {'bbox_inches':'tight', 'pad_inches':0.2}
plotfiletype='png'

In [5]:
savefig = True
zoom = True
if zoom:
    xmin, xmax = (-126.5,-121)
    ymin, ymax = (35.5,38.5)
    levels = np.linspace(-.1,.2,31)
else:
    xmin, xmax = (-131,-121)
    ymin, ymax = (34, 43)
    levels = np.linspace(-.3,.3,41)

In [6]:
def plot_SSH_map(tind):
    plt.clf()
    ax = plt.axes(projection = ccrs.PlateCarree(central_longitude=200))  # Orthographic
    extent = [xmin, xmax, ymin, ymax]
    day_str = np.datetime_as_string(ds.time[tind], unit='D')
    ax.set_extent(extent, crs=ccrs.PlateCarree())
    ax.set_title('Sea level anomaly (DUACS), '+ day_str,size = 10.)

    #plt.set_cmap(cmap=plt.get_cmap('nipy_spectral'))
    plt.set_cmap(cmap=plt.get_cmap('turbo'))
    gl = ax.gridlines(draw_labels=True, dms=True, x_inline=False, y_inline=False, alpha=0.5, linestyle='--')
    gl.top_labels = False
    gl.right_labels = False
    #gl.xlocator = matplotlib.ticker.MaxNLocator(10)
    #gl.xlocator = matplotlib.ticker.AutoLocator
    # gl.xlocator = matplotlib.ticker.FixedLocator(np.arange(0, 360 ,30))

    cs = ax.contourf(ds.longitude,ds.latitude,np.squeeze(ds.sla.isel(time=tind)), levels, extend='both', transform=ccrs.PlateCarree())
    # cs = ax.pcolormesh(ds.longitude,ds.latitude,np.squeeze(ds.sla), vmin=levels[0], vmax=levels[-1], transform=ccrs.PlateCarree())
    # cb = plt.colorbar(cs,ax=ax,shrink=.8,pad=.05)
    cb = plt.colorbar(cs,fraction = 0.022,extend='both')
    cb.set_label('SLA [m]',fontsize = 10)
    ax.coastlines()
    ax.add_feature(cartopy.feature.LAND, zorder=3, facecolor=[.6,.6,.6], edgecolor='black')

    functions.plot_ops_area(ax,transform=ccrs.PlateCarree(),color='w')

    # Add a 10 km scale bar
    km_per_deg_lat=gsw.geostrophy.distance((121.7,121.7), (37,38))/1000
    deg_lat_equal_10km=10/km_per_deg_lat
    x0 = -122 
    y0 = 37
    ax.plot(x0+np.asarray([0, 0]),y0+np.asarray([0.,deg_lat_equal_10km[0]]),transform=ccrs.PlateCarree(),color='k',zorder=3)
    ax.text(x0+1/60, y0+.15/60, '10 km', fontsize=6,transform=ccrs.PlateCarree())

    u = np.squeeze(ds.ugos.isel(time=tind)) #dtype=object
    v = np.squeeze(ds.vgos.isel(time=tind))
    ax.quiver(ds.longitude.values,ds.latitude.values, u.values, v.values,  scale=5, transform=ccrs.PlateCarree())
    x0 = -122.5 
    y0 = 38.5
    ax.quiver(np.array([x0]), np.array([y0]), -np.array([0.25/np.sqrt(2)],), np.array([0.25/np.sqrt(2)]), scale=3, transform=ccrs.PlateCarree(),zorder=3)
    ax.text(x0+3/60, y0+.15/60, '0.25 m/s', fontsize=6,transform=ccrs.PlateCarree())

    if savefig:
        plt.savefig(__figdir__+'SLA'+str(tind)+'.'+plotfiletype,**savefig_args)

In [7]:
# url = 'https://nrt.cmems-du.eu/thredds/dodsC/dataset-duacs-nrt-global-merged-allsat-phy-l4'
file = '../data/external/nrt_global_swonc_phy_l3_1hz_20230330_20230402.nc'
ds = xr.open_dataset(file)

In [17]:
plt.figure()
vmin = levels[0]
vmax = levels[-1]
plt.set_cmap(cmap=plt.get_cmap('turbo'))
cs = plt.scatter(ds.longitude,ds.latitude,s=None,c=ds.sla_unfiltered,vmin=levels[0],vmax=levels[-1])
cb = plt.colorbar(cs,fraction = 0.022,extend='both')